In [1]:
%pip install -r requirements.txt
%pip install nltk
%pip install numpy
%pip install gensim.downloader
%pip install nltk.tokenize
%pip install gensim
%pip install datasets
%pip install torch
%pip install torch.utils.data
%pip install torch.nn
%pip install torch.nn.functional
%pip install torch.optim
%pip install torchtext

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gensim.downloader (from versions: none)
ERROR: No matching distribution found for gensim.downloader


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch.nn (from versions: none)
ERROR: No matching distribution found for torch.nn


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch.nn.functional (from versions: none)
ERROR: No matching distribution found for torch.nn.functional


   ---------------------------------------- 0.0/59.1 MB ? eta -:--:--
   -------- ------------------------------- 12.8/59.1 MB 67.3 MB/s eta 0:00:01
   -------------------- ------------------- 29.6/59.1 MB 72.3 MB/s eta 0:00:01
   ------------------------------- -------- 46.9/59.1 MB 76.6 MB/s eta 0:00:01
   ---------------------------------------  59.0/59.1 MB 75.2 MB/s eta 0:00:01
   ---------------------------------------- 59.1/59.1 MB 58.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 30.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 51.4 MB/s eta 0:00:00

   ---------------------------------------- 0/7 [pillow]
   ---------------------------------------- 0/7 [pillow]
   ---------------------------------------- 0/7 [pillow]
   ---------------------------------------- 0/7 [pillow]
   -----------

# A Clear Structure of the Workflow

Objective: build a clean preprocessing pipeline for vocabulary, tokenization, pre-trained vector embedding, dataset and dataloader, and model architecture.

Note: we don't remove punctuation. It might be a feature for prediction.

1. Build vocabulary by first using NLTK tokenizer. Identify which words are in our training dataset is in our word2vec model, and which aren't. Those in the word2vec model will have its embedding readily available. Those outside will be mapped to OOV token, initialized by zero vector. The vocabulary will have 0-th token reserved for padding token and the OOV token will be the 1-th token. The rest of the words will fill the rest. The result should be a dictionaries word2idx and idx2word.
2. Now we need to handle preprocessing by custom dataset and dataloader. It should accept train_dataset['text'], convert it into tensor of indices from word2idx (it should map oov words correctly), and pad it using the pad token with a defined MAX_LENGTH and pair it with train_dataset['label']. The dataloader should be constructed with collate_fn to construct the appropriate batch.
3. The RNN should be defined. It first takes the indices from the batch given by the dataloader, embed it using pre-trained word2vec model (but we keep the embedding matrix frozen), and pass it to the RNN. However, we want to make sure that we don't use pad token as a feature. The final output should be sigmoid activated neuron so that it can be used for training with BCELoss.

In [2]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import gensim.downloader as api
import numpy as np

# Load pre-trained Word2vec or GloVe model (example uses Word2vec)
word_vectors = api.load("word2vec-google-news-300")  # or your chosen GloVe model
embedding_dim = word_vectors.vector_size

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\j_fel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Load dataset
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

c:\Users\j_fel\OneDrive\Desktop\NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Part 1

In [4]:
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(validation_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Training samples: 8530
Validation samples: 1066
Test samples: 1066


In [5]:
vocabulary = set()
for entry in train_dataset:
    words = entry['text'].split()
    vocabulary.update(words)

print(f"Vocabulary size: {len(vocabulary)}")

Vocabulary size: 18951


In [6]:

embedding_dimension = word_vectors.vector_size

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((len(vocabulary), embedding_dimension))

# Create a dictionary mapping each word in the vocabulary to a unique index
word_to_idx = {word: idx for idx, word in enumerate(vocabulary)}


oov_words = []

# Populate the embedding matrix with embeddings for each word in the vocabulary
for word, idx in word_to_idx.items():
    if word in word_vectors:
        embedding_matrix[idx] = word_vectors[word]
    else:
        oov_words.append(word)


print(f"OOV words count: {len(oov_words)}")

OOV words count: 4585


In [7]:
for word in oov_words:
    idx = word_to_idx[word]
    embedding_matrix[idx] = np.zeros(embedding_dimension)  # Zero vector for OOV words

print("Completed embedding matrix with OOV handling.")

Completed embedding matrix with OOV handling.


# Part 2

In [8]:
# Initialize special tokens and indices
PAD_TOKEN = "<PAD>"
OOV_TOKEN = "<OOV>"
PAD_IDX = 0
OOV_IDX = 1

# Initialize vocabulary and embeddings dictionary
word2idx = {PAD_TOKEN: PAD_IDX, OOV_TOKEN: OOV_IDX}
idx2word = {PAD_IDX: PAD_TOKEN, OOV_IDX: OOV_TOKEN}

# Create an embedding matrix initialized with zero for special tokens
embedding_matrix = [np.zeros(embedding_dim), np.zeros(embedding_dim)]

# Function to build vocabulary and embedding matrix
def build_vocabulary(sentences):
    for sentence in sentences:
        tokens = [token.lower() for token in word_tokenize(sentence)]
        for token in tokens:
            if token not in word2idx:
                if token in word_vectors:
                    word2idx[token] = len(word2idx)
                    idx2word[len(idx2word)] = token
                    embedding_matrix.append(word_vectors[token])
                #else:
                #   word2idx[token] = OOV_IDX  # Map OOV tokens to <OOV>

    return word2idx, idx2word, np.array(embedding_matrix)

word2idx, idx2word, embedding_matrix = build_vocabulary(train_dataset['text'])

# Verifying the results
print("Vocabulary:", word2idx)
print("Embedding matrix shape:", embedding_matrix.shape)

# To get the word with OOV handling, call
word2idx.get("unknown_word", OOV_IDX)

Vocabulary: {'<PAD>': 0, '<OOV>': 1, 'the': 2, 'rock': 3, 'is': 4, 'destined': 5, 'be': 6, 'century': 7, 'new': 8, 'conan': 9, 'that': 10, 'he': 11, 'going': 12, 'make': 13, 'splash': 14, 'even': 15, 'greater': 16, 'than': 17, 'arnold': 18, 'schwarzenegger': 19, 'van': 20, 'or': 21, 'steven': 22, 'gorgeously': 23, 'elaborate': 24, 'continuation': 25, 'lord': 26, 'rings': 27, 'trilogy': 28, 'so': 29, 'huge': 30, 'column': 31, 'words': 32, 'can': 33, 'not': 34, 'adequately': 35, 'describe': 36, 'co-writer/director': 37, 'peter': 38, 'jackson': 39, 'expanded': 40, 'vision': 41, 'j': 42, 'r': 43, 'effective': 44, 'but': 45, 'biopic': 46, 'if': 47, 'you': 48, 'sometimes': 49, 'like': 50, 'go': 51, 'movies': 52, 'have': 53, 'fun': 54, 'wasabi': 55, 'good': 56, 'place': 57, 'start': 58, 'emerges': 59, 'as': 60, 'something': 61, 'rare': 62, 'an': 63, 'issue': 64, 'movie': 65, 'honest': 66, 'keenly': 67, 'observed': 68, 'it': 69, 'does': 70, "n't": 71, 'feel': 72, 'one': 73, 'film': 74, 'provid

1

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Constants
MAX_LENGTH = 50  # Maximum length for each sequence (adjustable)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, word2idx, max_length=MAX_LENGTH):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and convert text to indices
        tokens = word_tokenize(self.texts[idx].lower())
        indices = [self.word2idx.get(token, OOV_IDX) for token in tokens]  # Map OOV words to OOV_IDX

        # Store actual length before padding
        length = min(len(indices), self.max_length)

        # Pad or truncate to max_length
        if len(indices) < self.max_length:
            indices.extend([PAD_IDX] * (self.max_length - len(indices)))
        else:
            indices = indices[:self.max_length]

        # Convert label to tensor
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        return torch.tensor(indices, dtype=torch.long), label, length

# Collate function for DataLoader
def collate_fn(batch):
    texts, labels, lengths = zip(*batch)
    texts = torch.stack(texts)
    labels = torch.stack(labels)
    lengths = torch.tensor(lengths)
    return texts, labels, lengths


In [10]:
custom_train_dataset = SentimentDataset(train_dataset['text'], train_dataset['label'], word2idx)
custom_dataloader = DataLoader(custom_train_dataset, batch_size=64, collate_fn=collate_fn, shuffle=True)

In [11]:
a, b, c = next(iter(custom_dataloader))
a, b, c

(tensor([[   1,  334,    1,  ...,    0,    0,    0],
         [ 289,   34,   79,  ...,    0,    0,    0],
         [  69,    1,   73,  ...,    0,    0,    0],
         ...,
         [   2, 9574, 4141,  ...,    0,    0,    0],
         [   2,  359,   10,  ...,    0,    0,    0],
         [   2,  776,   65,  ...,    0,    0,    0]]),
 tensor([1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
         0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
         0., 1., 0., 1., 0., 1., 1., 1., 0., 1.]),
 tensor([ 9, 21, 22, 17, 13, 21,  9, 23, 29, 19,  4, 25, 17, 16, 24, 26, 16, 26,
         33, 41, 27,  5, 16, 23,  7, 33, 30, 17, 16, 12, 28, 33, 24, 15, 35, 45,
         18, 19, 22, 13,  8, 36, 26, 19, 26, 45, 14, 27,  3, 12,  4,  9,  7, 13,
         23, 27, 14,  2, 34, 29, 19, 26, 29,  9]))

In [12]:
import torch
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size=1, num_layers=1, dropout=0.5):
        super(SentimentRNN, self).__init__()

        # Embedding layer using the pre-loaded Word2Vec embedding matrix
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32),
                                                      freeze=True, padding_idx=PAD_IDX)

        # RNN layer (LSTM in this case)
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        # x is the batch of token indices; lengths is the actual lengths before padding
        embedded = self.embedding(x)  # [batch_size, seq_length, embedding_dim]

        # Pack padded batch of sequences for RNN to skip over padding tokens
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)

        # RNN forward pass
        packed_output, (hidden, _) = self.rnn(packed_embedded)

        # Take the hidden state from the last RNN layer
        hidden = hidden[-1]  # [batch_size, hidden_size]

        # Fully connected layer and sigmoid activation
        out = self.fc(hidden)
        return self.sigmoid(out).squeeze()

# Define hyperparameters
hidden_size = 128  # Number of features in the hidden state
output_size = 1  # Output size for binary classification (sentiment)
num_layers = 1   # Number of RNN layers
dropout = 0.5    # Dropout rate

# Initialize the model with the pre-loaded embedding matrix from Word2Vec
model = SentimentRNN(embedding_matrix, hidden_size, output_size, num_layers, dropout)

In [13]:
custom_test_dataset = SentimentDataset(test_dataset['text'], test_dataset['label'], word2idx)
custom_test_dataloader = DataLoader(custom_test_dataset, batch_size=64, collate_fn=collate_fn)

import torch.optim as optim
from tqdm import tqdm

def train_and_evaluate(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs, device):
    model = model.to(device)

    for epoch in range(num_epochs):
        # Set model to training mode
        model.train()
        epoch_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Training Loop
        for texts, labels, lengths in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
            # Move data to the device
            texts, labels, lengths = texts.to(device), labels.to(device), lengths.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(texts, lengths)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Update loss and accuracy
            epoch_loss += loss.item() * texts.size(0)  # Multiply by batch size to get total loss for the batch
            predictions = (outputs >= 0.5).float()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

        # Calculate average training loss and accuracy for the epoch
        avg_train_loss = epoch_loss / len(train_dataloader.dataset)
        train_accuracy = correct_predictions / total_predictions
        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Evaluate on test set at the end of each epoch
        test_loss, test_accuracy = evaluate_on_test_set(model, test_dataloader, criterion, device)
        print(f"Epoch {epoch + 1}/{num_epochs} - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

def evaluate_on_test_set(model, test_dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient computation
        for texts, labels, lengths in test_dataloader:
            # Move data to the device
            texts, labels, lengths = texts.to(device), labels.to(device), lengths.to(device)

            # Forward pass
            outputs = model(texts, lengths)
            loss = criterion(outputs, labels)

            # Update loss and accuracy
            total_loss += loss.item() * texts.size(0)  # Multiply by batch size for total batch loss
            predictions = (outputs >= 0.5).float()  # Convert probabilities to binary predictions
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(test_dataloader.dataset)
    accuracy = correct_predictions / total_predictions

    return avg_loss, accuracy

# Set device, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # Set the number of epochs as needed

# Start training and evaluating
train_and_evaluate(model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs, device)

Epoch 1/10 - Training: 100%|██████████| 134/134 [00:11<00:00, 11.51it/s]


Epoch 1/10 - Train Loss: 0.5750, Train Accuracy: 0.7018
Epoch 1/10 - Test Loss: 0.5008, Test Accuracy: 0.7636


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:10<00:00, 12.48it/s]


Epoch 2/10 - Train Loss: 0.4763, Train Accuracy: 0.7782
Epoch 2/10 - Test Loss: 0.4891, Test Accuracy: 0.7608


Epoch 3/10 - Training: 100%|██████████| 134/134 [00:13<00:00, 10.18it/s]


...

Epoch 8/10 - Train Loss: 0.3490, Train Accuracy: 0.8450
Epoch 8/10 - Test Loss: 0.4747, Test Accuracy: 0.7767


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:09<00:00, 13.51it/s]


Epoch 9/10 - Train Loss: 0.3258, Train Accuracy: 0.8572
Epoch 9/10 - Test Loss: 0.4963, Test Accuracy: 0.7777


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:11<00:00, 11.87it/s]


Epoch 10/10 - Train Loss: 0.3024, Train Accuracy: 0.8700
Epoch 10/10 - Test Loss: 0.4738, Test Accuracy: 0.7805


# Part 3.1

In [14]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size=1, num_layers=1, dropout=0.5):
        super(SentimentRNN, self).__init__()

        # Embedding layer without freezing, so embeddings are trainable
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=False,  # Set to False to allow training the embeddings
            padding_idx=PAD_IDX
        )

        # RNN layer (LSTM)
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        # x is the batch of token indices; lengths is the actual lengths before padding
        embedded = self.embedding(x)  # [batch_size, seq_length, embedding_dim]

        # Pack padded batch of sequences for RNN to skip over padding tokens
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)

        # RNN forward pass
        packed_output, (hidden, _) = self.rnn(packed_embedded)

        # Take the hidden state from the last RNN layer
        hidden = hidden[-1]  # [batch_size, hidden_size]

        # Fully connected layer and sigmoid activation
        out = self.fc(hidden)
        return self.sigmoid(out).squeeze()

In [15]:
# Ensure embeddings are trainable in SentimentRNN
model = SentimentRNN(embedding_matrix, hidden_size=128, output_size=1, num_layers=1, dropout=0.5)

# Set up training configurations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # or any other desired number of epochs

# Train and evaluate the model, which will print test accuracy after each epoch
train_and_evaluate(model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs, device)


Epoch 1/10 - Training: 100%|██████████| 134/134 [00:14<00:00,  9.49it/s]


Epoch 1/10 - Train Loss: 0.5461, Train Accuracy: 0.7124
Epoch 1/10 - Test Loss: 0.4713, Test Accuracy: 0.7692


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:14<00:00,  9.03it/s]


Epoch 2/10 - Train Loss: 0.3215, Train Accuracy: 0.8676
Epoch 2/10 - Test Loss: 0.4400, Test Accuracy: 0.7795


Epoch 3/10 - Training: 100%|██████████| 134/134 [00:14<00:00,  9.37it/s]


...

Epoch 8/10 - Train Loss: 0.0178, Train Accuracy: 0.9937
Epoch 8/10 - Test Loss: 0.9450, Test Accuracy: 0.7542


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:13<00:00,  9.58it/s]


Epoch 9/10 - Train Loss: 0.0189, Train Accuracy: 0.9937
Epoch 9/10 - Test Loss: 1.4423, Test Accuracy: 0.7533


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:12<00:00, 10.74it/s]


Epoch 10/10 - Train Loss: 0.0108, Train Accuracy: 0.9962
Epoch 10/10 - Test Loss: 1.4376, Test Accuracy: 0.7645


# Part 3.2

In [16]:
def build_vocabulary(sentences):
    # Initialize within the function scope to avoid external influence
    word2idx = {PAD_TOKEN: PAD_IDX, OOV_TOKEN: OOV_IDX}
    idx2word = {PAD_IDX: PAD_TOKEN, OOV_IDX: OOV_TOKEN}
    embedding_matrix = [np.zeros(embedding_dim), np.zeros(embedding_dim)]  # Start with PAD and OOV vectors

    for sentence in sentences:
        tokens = [token.lower() for token in word_tokenize(sentence)]
        for token in tokens:
            if token not in word2idx:
                if token in word_vectors:
                    # Add embedding for token in word_vectors
                    word2idx[token] = len(word2idx)
                    idx2word[len(idx2word)] = token
                    embedding_matrix.append(word_vectors[token])
                else:
                    # Map OOV words to the existing <OOV> token without adding them to the vocabulary
                    # No action needed since __getitem__ in Dataset handles OOV mapping
                    continue

    # Convert embedding_matrix to a numpy array after completion
    embedding_matrix = np.array(embedding_matrix)

    # Debugging information
    print("Final word2idx size:", len(word2idx))
    print("Final embedding_matrix size:", embedding_matrix.shape[0])

    # Ensure embedding matrix size matches vocabulary size
    assert len(word2idx) == embedding_matrix.shape[0], (
        f"Mismatch between word2idx ({len(word2idx)}) and embedding_matrix size ({embedding_matrix.shape[0]})"
    )

    return word2idx, idx2word, embedding_matrix

# Rebuild vocabulary and embedding matrix with isolated initialization
word2idx, idx2word, embedding_matrix = build_vocabulary(train_dataset['text'])

# Reinitialize Datasets with updated word2idx
custom_train_dataset = SentimentDataset(train_dataset['text'], train_dataset['label'], word2idx)
custom_test_dataset = SentimentDataset(test_dataset['text'], test_dataset['label'], word2idx)

# Reinitialize DataLoaders with updated Datasets
custom_dataloader = DataLoader(custom_train_dataset, batch_size=64, collate_fn=collate_fn, shuffle=True)
custom_test_dataloader = DataLoader(custom_test_dataset, batch_size=64, collate_fn=collate_fn)

Final word2idx size: 14419
Final embedding_matrix size: 14419


In [17]:
# Ensure embeddings are set up with zero initialization for OOV words in SentimentRNN
model = SentimentRNN(embedding_matrix, hidden_size=128, output_size=1, num_layers=1, dropout=0.5)

# Training configurations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10  # or another preferred number of epochs

# Train and evaluate the model
train_and_evaluate(model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs, device)


Epoch 1/10 - Training:   0%|          | 0/134 [00:00<?, ?it/s]

Epoch 1/10 - Training: 100%|██████████| 134/134 [00:14<00:00,  9.21it/s]


Epoch 1/10 - Train Loss: 0.5527, Train Accuracy: 0.7097
Epoch 1/10 - Test Loss: 0.4510, Test Accuracy: 0.7936


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:18<00:00,  7.39it/s]


Epoch 2/10 - Train Loss: 0.3259, Train Accuracy: 0.8669
Epoch 2/10 - Test Loss: 0.4566, Test Accuracy: 0.7842


...

Epoch 8/10 - Train Loss: 0.0243, Train Accuracy: 0.9921
Epoch 8/10 - Test Loss: 1.1315, Test Accuracy: 0.7636


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:16<00:00,  8.20it/s]


Epoch 9/10 - Train Loss: 0.0110, Train Accuracy: 0.9962
Epoch 9/10 - Test Loss: 1.3195, Test Accuracy: 0.7486


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:17<00:00,  7.84it/s]


Epoch 10/10 - Train Loss: 0.0049, Train Accuracy: 0.9987
Epoch 10/10 - Test Loss: 1.4228, Test Accuracy: 0.7683


# Part 3.3 - Implementing biLSTM and biGRU Models

### biLSTM Model

In [18]:
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size=1, num_layers=2, dropout=0.5):
        super(SentimentBiLSTM, self).__init__()

        # Embedding layer with trainable embeddings
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=False, padding_idx=PAD_IDX
        )

        # biLSTM layer with multiple layers
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed_embedded)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)  # Concatenate final hidden states from both directions
        out = self.fc(hidden)
        return self.sigmoid(out).squeeze()

In [19]:
print("Training and evaluating biLSTM model")
biLSTM_model = SentimentBiLSTM(embedding_matrix, hidden_size=128)
optimizer = torch.optim.Adam(biLSTM_model.parameters(), lr=0.001)
train_and_evaluate(biLSTM_model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs=10, device=device)

Training and evaluating biLSTM model


Epoch 1/10 - Training:   0%|          | 0/134 [00:00<?, ?it/s]

Epoch 1/10 - Training: 100%|██████████| 134/134 [00:49<00:00,  2.73it/s]


Epoch 1/10 - Train Loss: 0.5337, Train Accuracy: 0.7019
Epoch 1/10 - Test Loss: 0.4397, Test Accuracy: 0.8021


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:44<00:00,  3.04it/s]


...

Epoch 8/10 - Train Loss: 0.0189, Train Accuracy: 0.9937
Epoch 8/10 - Test Loss: 1.3125, Test Accuracy: 0.7674


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:41<00:00,  3.22it/s]


Epoch 9/10 - Train Loss: 0.0085, Train Accuracy: 0.9971
Epoch 9/10 - Test Loss: 1.3973, Test Accuracy: 0.7664


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:44<00:00,  3.01it/s]


Epoch 10/10 - Train Loss: 0.0104, Train Accuracy: 0.9968
Epoch 10/10 - Test Loss: 1.3669, Test Accuracy: 0.7692


### biGRU Model

In [20]:
class SentimentBiGRU(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size=1, num_layers=2, dropout=0.5):
        super(SentimentBiGRU, self).__init__()

        # Embedding layer with trainable embeddings
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=False, padding_idx=PAD_IDX
        )

        # biGRU layer with multiple layers
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers,
                          batch_first=True, bidirectional=True, dropout=dropout)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size * 2, output_size)  # Multiply by 2 for bidirectional

        # Sigmoid activation for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, hidden = self.gru(packed_embedded)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)  # Concatenate final hidden states from both directions
        out = self.fc(hidden)
        return self.sigmoid(out).squeeze()

In [21]:
print("Training and evaluating biGRU model")
biGRU_model = SentimentBiGRU(embedding_matrix, hidden_size=128)
optimizer = torch.optim.Adam(biGRU_model.parameters(), lr=0.001)
train_and_evaluate(biGRU_model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs=10, device=device)

Training and evaluating biGRU model


Epoch 1/10 - Training: 100%|██████████| 134/134 [00:37<00:00,  3.59it/s]


Epoch 1/10 - Train Loss: 0.5168, Train Accuracy: 0.7358
Epoch 1/10 - Test Loss: 0.4469, Test Accuracy: 0.8021


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:39<00:00,  3.42it/s]


Epoch 2/10 - Train Loss: 0.2946, Train Accuracy: 0.8755
Epoch 2/10 - Test Loss: 0.4514, Test Accuracy: 0.7880


...

Epoch 8/10 - Train Loss: 0.0079, Train Accuracy: 0.9975
Epoch 8/10 - Test Loss: 1.3603, Test Accuracy: 0.7889


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:48<00:00,  2.79it/s]


Epoch 9/10 - Train Loss: 0.0118, Train Accuracy: 0.9950
Epoch 9/10 - Test Loss: 1.7621, Test Accuracy: 0.7561


Epoch 10/10 - Training: 100%|██████████| 134/134 [01:06<00:00,  2.02it/s]


Epoch 10/10 - Train Loss: 0.0111, Train Accuracy: 0.9958
Epoch 10/10 - Test Loss: 1.5038, Test Accuracy: 0.7617


# Part 3.4 - Implementing CNN for Sentence Representation

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    def __init__(self, embedding_matrix, num_filters=100, filter_sizes=[3,4,5], hidden_dim=128, output_size=1, dropout=0.5):
        super(SentimentCNN, self).__init__()

        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=False,  # Embeddings are trainable
            padding_idx=PAD_IDX
        )

        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embedding_dim)) for fs in filter_sizes
        ])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        embedded = self.embedding(x)  # [batch_size, seq_length, embedding_dim]
        embedded = embedded.unsqueeze(1)  # [batch_size, 1, seq_length, embedding_dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]  # List of [batch_size, num_filters, *]
        pooled = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in conved]  # List of [batch_size, num_filters]

        cat = torch.cat(pooled, dim=1)  # [batch_size, num_filters * len(filter_sizes)]
        dropped = self.dropout(cat)
        out = self.fc(dropped)
        return self.sigmoid(out).squeeze()


In [23]:
print("Training and evaluating CNN model")
cnn_model = SentimentCNN(embedding_matrix, num_filters=100, filter_sizes=[3,4,5], hidden_dim=128, dropout=0.5)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)
train_and_evaluate(cnn_model, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs=10, device=device)

Training and evaluating CNN model


Epoch 1/10 - Training:   0%|          | 0/134 [00:00<?, ?it/s]

Epoch 1/10 - Training: 100%|██████████| 134/134 [00:20<00:00,  6.38it/s]


Epoch 1/10 - Train Loss: 0.5380, Train Accuracy: 0.7314
Epoch 1/10 - Test Loss: 0.4279, Test Accuracy: 0.8002


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:18<00:00,  7.26it/s]


...

Epoch 8/10 - Train Loss: 0.0061, Train Accuracy: 0.9991
Epoch 8/10 - Test Loss: 0.8684, Test Accuracy: 0.7786


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:16<00:00,  7.90it/s]


Epoch 9/10 - Train Loss: 0.0039, Train Accuracy: 0.9996
Epoch 9/10 - Test Loss: 0.9156, Test Accuracy: 0.7749


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:15<00:00,  8.67it/s]


Epoch 10/10 - Train Loss: 0.0028, Train Accuracy: 0.9996
Epoch 10/10 - Test Loss: 1.0467, Test Accuracy: 0.7711


# Part 3.5

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SentimentRNNWithAttention(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size=1, num_layers=1, dropout=0.5):
        super(SentimentRNNWithAttention, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False, padding_idx=PAD_IDX)

        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0, bidirectional=True)

        self.attention = nn.Linear(hidden_size * 2, 1)  # Multiply by 2 for bidirectional
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.rnn(packed_embedded)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        attention_weights = F.softmax(self.attention(output), dim=1)
        context_vector = torch.sum(attention_weights * output, dim=1)

        out = self.fc(context_vector)
        return self.sigmoid(out).squeeze()


In [25]:
print("Training and evaluating biLSTM model with Attention")
biLSTM_model_with_Attention = SentimentRNNWithAttention(embedding_matrix, hidden_size, output_size, num_layers, dropout)
optimizer = torch.optim.Adam(biLSTM_model_with_Attention.parameters(), lr=0.001)
train_and_evaluate(biLSTM_model_with_Attention, custom_dataloader, custom_test_dataloader, criterion, optimizer, num_epochs=10, device=device)


Training and evaluating biLSTM model with Attention


Epoch 1/10 - Training: 100%|██████████| 134/134 [00:28<00:00,  4.72it/s]


Epoch 1/10 - Train Loss: 0.5692, Train Accuracy: 0.7158
Epoch 1/10 - Test Loss: 0.4671, Test Accuracy: 0.7739


Epoch 2/10 - Training: 100%|██████████| 134/134 [00:31<00:00,  4.24it/s]


Epoch 2/10 - Train Loss: 0.3428, Train Accuracy: 0.8576
Epoch 2/10 - Test Loss: 0.4495, Test Accuracy: 0.7842


...

Epoch 8/10 - Train Loss: 0.0063, Train Accuracy: 0.9979
Epoch 8/10 - Test Loss: 1.0911, Test Accuracy: 0.7917


Epoch 9/10 - Training: 100%|██████████| 134/134 [00:35<00:00,  3.75it/s]


Epoch 9/10 - Train Loss: 0.0034, Train Accuracy: 0.9987
Epoch 9/10 - Test Loss: 1.3379, Test Accuracy: 0.7795


Epoch 10/10 - Training: 100%|██████████| 134/134 [00:30<00:00,  4.35it/s]


Epoch 10/10 - Train Loss: 0.0032, Train Accuracy: 0.9993
Epoch 10/10 - Test Loss: 1.2993, Test Accuracy: 0.7880
